# Datos de Voz

**Para los datos de voz usamos el DataSet de "The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS )" obtenido en la pagina "https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio"**

Con el fin de utilizarlos de una mejor manera se obtuvieron las etiquetas de las categorias de cada audio como especifica el autor.

Cada uno de los 1440 archivos tiene un nombre de archivo único. El nombre del archivo consta de un identificador numérico de 7 partes (p. ej., 03-01-06-01-02-01-12.wav). Estos identificadores definen las características del estímulo:

Identificadores de nombre de archivo

* Modalidad (01 = full-AV, 02 = solo video, 03 = solo audio).
* Canal vocal (01 = habla, 02 = canción).
* Emoción (01 = neutral, 02 = tranquilo, 03 = feliz, 04 = triste, 05 = enojado, 06 = temeroso, 07 = asco, 08 = sorprendido).
* Intensidad emocional (01 = normal, 02 = fuerte). NOTA: No hay una intensidad fuerte para la emoción 'neutral'.
* Declaración (01 = "Los niños hablan junto a la puerta", 02 = "Los perros están sentados junto a la puerta").
* Repetición (01 = 1ª repetición, 02 = 2ª repetición).
* Actor (del 01 al 24. Los actores impares son hombres, los actores pares son mujeres).

En el siguiente codigo vamos a almacenar los datos en un archivo Json para poder utilizar los espectrogramas sin tener que volver a cargar los audios, usamos scipy.interpolate para tener los datos del espectrograma de un mismo tamaño.


In [1]:
import os
import json
import librosa
import numpy as np
from scipy.interpolate import interp1d

# Establecer la ruta de la carpeta que contiene los archivos de audio
ruta_carpeta = "archive/"
print(os.listdir(ruta_carpeta)[:-1])
# Crear una lista vacía para almacenar los datos del espectrograma de cada archivo de audio
espectrograma = []
max_length = 240

# Recorrer cada archivo de audio en la carpeta
for carpeta in os.listdir(ruta_carpeta)[:-1]:
    ruta_carpeta_audio = ruta_carpeta+carpeta+"/"
    print(ruta_carpeta_audio)
    for archivo_audio in os.listdir(ruta_carpeta_audio):
        # Comprobar que el archivo es un archivo de audio
        if archivo_audio.endswith(".wav"):
            # Obtener las etiquetas de categoría del archivo de audio a partir del nombre del archivo
            etiquetas = archivo_audio.split("-")
            Modality = etiquetas[0]  # 01-full, 02-Video, 03-audio
            Vocal = etiquetas[1]  # 01-speech, 02-song
            Emotion = etiquetas[2]  # 01-neutral, 02-calm, 03-happy, 04-sad, 05-angry, 06-fearful, 07-disgust, 08-surprised
            Intensity = etiquetas[3]  # 01-normal, 02-strong
            Statement = etiquetas[4]  # 01-"Kids are talking by the door", 02-"Dogs are sitting by the door").
            Repetition = etiquetas[5] # 01-1st repetition, 02-2nd repetition).
            Actor = etiquetas[6].split(".")[0] # Par-Mujer, Impar-Hombre

            # Cargar el archivo de audio y extraer los espectrogramas
            ruta_archivo_audio = os.path.join(ruta_carpeta_audio, archivo_audio)
            y, sr = librosa.load(ruta_archivo_audio,sr=22050)
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            f = interp1d(np.linspace(0, 1, mel_spec_db.shape[1]), mel_spec_db, kind='linear', axis=1)
            mel_spec_db = f(np.linspace(0, 1, max_length))

            # Agregar los datos del espectrograma a la lista junto con las etiquetas de categoría
            espectrograma.append({
                "Modality": Modality,
                "Vocal": Vocal,
                "Emotion": Emotion,
                "Intensity": Intensity,
                "Statement": Statement,
                "Repetition": Repetition,
                "Actor": Actor,
                "Espectrograma": mel_spec_db.tolist()
            })

# Guardar los datos del espectrograma en un archivo JSON
ruta_archivo_json = "Espectrograma.json"
with open(ruta_archivo_json, "w") as archivo_json:
    json.dump(espectrograma, archivo_json)


['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']
archive/Actor_01/
archive/Actor_02/
archive/Actor_03/
archive/Actor_04/
archive/Actor_05/
archive/Actor_06/
archive/Actor_07/
archive/Actor_08/
archive/Actor_09/
archive/Actor_10/
archive/Actor_11/
archive/Actor_12/
archive/Actor_13/
archive/Actor_14/
archive/Actor_15/
archive/Actor_16/
archive/Actor_17/
archive/Actor_18/
archive/Actor_19/
archive/Actor_20/
archive/Actor_21/
archive/Actor_22/
archive/Actor_23/
archive/Actor_24/


# Carga de datos

**Cargamos los datos y los guardamos en una variable llamada data**

In [3]:
import os
import json
# Crear datos de entrenamiento
ruta_archivo_json =  "Espectrograma.json"
with open(ruta_archivo_json, "r") as archivo_json:
    data = json.load(archivo_json)
print(len(data))

1440


# Datos

**Guardamos los datos en un DataFrame con la finalidad de poder usarlos de una manera mas facil**

Tambien hacemos un acomodo aleatorio de los datos y normalizamos los datos del espectrograma

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame()
df["Modality"] = [int(data[val]['Modality'])-1 for val in range(len(data)) ]
df["Vocal"] = [int(data[val]['Vocal'])-1 for val in range(len(data)) ]
df["Emotion"] = [int(data[val]['Emotion'])-1 for val in range(len(data)) ]
df["Intensity"] = [int(data[val]['Intensity'])-1 for val in range(len(data)) ]
df["Statement"] = [int(data[val]['Statement'])-1 for val in range(len(data)) ]
df["Repetition"] = [int(data[val]['Repetition'])-1 for val in range(len(data)) ]
df["Actor"] = [int(data[val]['Actor'])-1 for val in range(len(data)) ]
df["Espectrograma"] = [np.array(data[val]['Espectrograma']) for val in range(len(data)) ]

df=df.sample(frac=1).reset_index(drop=True)
print(df)
df_normalized = df["Espectrograma"].apply(lambda x: (x - x.min()) / (x.max() - x.min()) * 2 - 1)
print(df_normalized)

      Modality  Vocal  Emotion  Intensity  Statement  Repetition  Actor  \
0            2      0        3          0          1           0      6   
1            2      0        2          1          0           0      6   
2            2      0        1          0          1           0      1   
3            2      0        7          0          0           0     17   
4            2      0        1          0          0           1      0   
...        ...    ...      ...        ...        ...         ...    ...   
1435         2      0        6          0          1           1     21   
1436         2      0        4          1          1           1     18   
1437         2      0        1          1          1           1     20   
1438         2      0        5          0          1           0     22   
1439         2      0        6          0          0           0     18   

                                          Espectrograma  
0     [[-77.2802505493164, -77.0967561410

# Preparacion de datos para entrenamiento

**Acomodamos los datos dejando un 20% para el test y del 80% de los datos resntante un 20% para la validacion, los datos restantes a esto seran los que usaremos para entrenar**

In [6]:
porcentaje = 0.8
total = len(df["Actor"])
split = porcentaje*total


data_c = np.array([np.asarray(x) for x in df['Espectrograma'][:int(split)]])
data_e = np.asarray(df["Actor"][:int(split)])

porcentaje = 0.8
total = len(data_c)
split2 = porcentaje*total

#Train data 
train_data = data_c[:int(split2)]
train_data = train_data.astype(np.float)
train_data = train_data.reshape(-1, 128, 240, 1)

train_labels = data_e[:int(split2)]
train_labels = np.asarray(train_labels)
train_labels = train_labels.astype(np.float)
train_labels = train_labels.reshape(-1, 1)

print("train_data shape: ",train_data.shape)
print("train_labels shape: ",train_labels.shape)

#Valid data
val_data = data_c[int(split2):]
val_data = val_data.astype(np.float)
val_data = val_data.reshape(-1, 128, 240, 1)
val_labels = data_e[int(split2):]
val_labels = val_labels.astype(np.float)
val_labels = val_labels.reshape(-1, 1)

print("val_data shape: ",val_data.shape)
print("val_labels shape: ",val_labels.shape)


#Test data
test_data = np.array([np.asarray(x) for x in df['Espectrograma'][int(split):]])
test_data = test_data.astype(np.float)
test_data = test_data.reshape(-1, 128, 240, 1)

test_labels = np.asarray(df["Actor"][int(split):])
test_labels = test_labels.astype(np.float)
test_labels = test_labels.reshape(-1, 1)

print("test_data shape: ",test_data.shape)
print("test_labels shape: ",test_labels.shape)


train_data shape:  (921, 128, 240, 1)
train_labels shape:  (921, 1)
val_data shape:  (231, 128, 240, 1)
val_labels shape:  (231, 1)
test_data shape:  (288, 128, 240, 1)
test_labels shape:  (288, 1)


C:\Users\manue\AppData\Local\Temp/ipykernel_13620/3622886832.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_data = train_data.astype(np.float)
C:\Users\manue\AppData\Local\Temp/ipykernel_13620/3622886832.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_labels = train_labels.astype(np.float)
C:\Users\manue\AppData\Loc

# Definimos la arquitectura para el modelo de Caracteristicas de Voz en personas

**Para las caracteristicas de usuario usaremos la siguiente arquitectura**

In [7]:
import tensorflow as tf

# Definir la arquitectura de la red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 240, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.08)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.04)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 238, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 119, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 117, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 59, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 117056)            0         
_________________________________________________________________
dropout (Dropout)            (None, 117056)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

# Compilación del modelo

In [8]:
# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo

In [10]:
# Entrenar el modelo

model.fit(train_data, 
          train_labels, 
          epochs=150, 
          steps_per_epoch = 12,
          validation_steps = 3,
          validation_data=(val_data, val_labels))

Epoch 1/150
12/12 [==============================] - 19s 2s/step - loss: 6.7599 - accuracy: 0.0619 - val_loss: 6.4823 - val_accuracy: 0.0519
Epoch 2/150
12/12 [==============================] - 20s 2s/step - loss: 6.6136 - accuracy: 0.0510 - val_loss: 6.3573 - val_accuracy: 0.0130
Epoch 3/150
12/12 [==============================] - 20s 2s/step - loss: 6.4445 - accuracy: 0.0521 - val_loss: 6.3031 - val_accuracy: 0.0130
Epoch 4/150
12/12 [==============================] - 20s 2s/step - loss: 6.2857 - accuracy: 0.0706 - val_loss: 6.2938 - val_accuracy: 0.0130
Epoch 5/150
12/12 [==============================] - 21s 2s/step - loss: 6.0046 - accuracy: 0.0999 - val_loss: 6.2722 - val_accuracy: 0.0303
Epoch 6/150
12/12 [==============================] - 21s 2s/step - loss: 5.5767 - accuracy: 0.2172 - val_loss: 6.2181 - val_accuracy: 0.0303
Epoch 7/150
12/12 [==============================] - 21s 2s/step - loss: 5.1606 - accuracy: 0.3018 - val_loss: 6.1603 - val_accuracy: 0.0303
Epoch 8/150
1

Epoch 59/150
12/12 [==============================] - 21s 2s/step - loss: 1.3454 - accuracy: 0.9989 - val_loss: 3.7540 - val_accuracy: 0.8225
Epoch 60/150
12/12 [==============================] - 21s 2s/step - loss: 1.3241 - accuracy: 0.9967 - val_loss: 3.6485 - val_accuracy: 0.8571
Epoch 61/150
12/12 [==============================] - 22s 2s/step - loss: 1.3076 - accuracy: 0.9902 - val_loss: 3.5646 - val_accuracy: 0.8961
Epoch 62/150
12/12 [==============================] - 21s 2s/step - loss: 1.2530 - accuracy: 0.9978 - val_loss: 3.6292 - val_accuracy: 0.7749
Epoch 63/150
12/12 [==============================] - 20s 2s/step - loss: 1.2504 - accuracy: 0.9946 - val_loss: 3.6894 - val_accuracy: 0.7706
Epoch 64/150
12/12 [==============================] - 21s 2s/step - loss: 1.2101 - accuracy: 0.9913 - val_loss: 3.4001 - val_accuracy: 0.9134
Epoch 65/150
12/12 [==============================] - 20s 2s/step - loss: 1.1634 - accuracy: 0.9967 - val_loss: 3.4543 - val_accuracy: 0.8312
Epoch 

12/12 [==============================] - 22s 2s/step - loss: 0.3958 - accuracy: 0.9967 - val_loss: 0.6394 - val_accuracy: 0.9481
Epoch 117/150
12/12 [==============================] - 21s 2s/step - loss: 0.3783 - accuracy: 0.9967 - val_loss: 0.8440 - val_accuracy: 0.9048
Epoch 118/150
12/12 [==============================] - 22s 2s/step - loss: 0.3834 - accuracy: 0.9989 - val_loss: 0.7789 - val_accuracy: 0.9351
Epoch 119/150
12/12 [==============================] - 21s 2s/step - loss: 0.3800 - accuracy: 0.9978 - val_loss: 0.7375 - val_accuracy: 0.9264
Epoch 120/150
12/12 [==============================] - 21s 2s/step - loss: 0.3620 - accuracy: 0.9989 - val_loss: 0.6612 - val_accuracy: 0.9524
Epoch 121/150
12/12 [==============================] - 21s 2s/step - loss: 0.3649 - accuracy: 0.9967 - val_loss: 0.7450 - val_accuracy: 0.9264
Epoch 122/150
12/12 [==============================] - 22s 2s/step - loss: 0.3589 - accuracy: 0.9946 - val_loss: 0.8030 - val_accuracy: 0.9221
Epoch 123/150

# Evaluación del Modelo

In [20]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

9/9 [==============================] - 1s 134ms/step - loss: 0.6719 - accuracy: 0.9062
Test accuracy: 0.90625


# Guardamos el modelo

In [23]:
model.save('Personas.h5')

In [25]:
num = 51
array = model.predict(test_data)[num]
print(np.where(array == max(array)), max(array),test_labels[num])
print(np.where(array == max(array))[0])


(array([16], dtype=int64),) 0.9944478 [16.]
[16]


# Datos para el modelo de Detección de Sentimientos

In [ ]:
porcentaje = 0.8
total = len(df["Emotion"])
split = porcentaje*total


data_c = np.array([np.asarray(x) for x in df['Espectrograma'][:int(split)]])
data_e = np.asarray(df["Emotion"][:int(split)])

porcentaje = 0.8
total = len(data_c)
split2 = porcentaje*total

#Train data 
train_data = data_c[:int(split2)]
train_data = train_data.astype(np.float)
train_data = train_data.reshape(-1, 128, 240, 1)

train_labels = data_e[:int(split2)]
train_labels = np.asarray(train_labels)
train_labels = train_labels.astype(np.float)
train_labels = train_labels.reshape(-1, 1)

print("train_data shape: ",train_data.shape)
print("train_labels shape: ",train_labels.shape)

#Valid data
val_data = data_c[int(split2):]
val_data = val_data.astype(np.float)
val_data = val_data.reshape(-1, 128, 240, 1)

val_labels = data_e[int(split2):]
val_labels = val_labels.astype(np.float)
val_labels = val_labels.reshape(-1, 1)

print("val_data shape: ",val_data.shape)
print("val_labels shape: ",val_labels.shape)


#Test data
test_data = np.array([np.asarray(x) for x in df['Espectrograma'][int(split):]])
test_data = test_data.astype(np.float)
test_data = test_data.reshape(-1, 128, 240, 1)

test_labels = np.asarray(df["Emotion"][int(split):])
test_labels = test_labels.astype(np.float)
test_labels = test_labels.reshape(-1, 1)

print("test_data shape: ",test_data.shape)
print("test_labels shape: ",test_labels.shape)

# Definimos la arquitectura para el modelo de Caracteristicas de emociones

In [ ]:
import tensorflow as tf

# Definir la arquitectura de la red neuronal
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 240, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='softmax')
])
model_2.summary()

# Compilamos el modelo

In [ ]:
# Compilar el modelo
model_2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo

In [ ]:
# Entrenar el modelo

model_2.fit(train_data, 
          train_labels, 
          epochs=150, 
          steps_per_epoch = 12,
          validation_steps = 3,
          validation_data=(val_data, val_labels))

# Evaluacion de Modelo

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model_2.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Guardamos el modelo

In [ ]:
model.save('emociones_Spec.h5')